In [1]:
import pandas as pd
import tensorflow as tf
from keras_preprocessing.image import load_img
from keras_preprocessing.image import img_to_array
from kaggle_datasets import KaggleDatasets
from numpy import save
from numpy import asarray
from os import listdir
from numpy import load
from tensorflow.keras.optimizers import RMSprop
import matplotlib.pyplot as plt
import random
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
!pip install ../input/sequencedata/efficientnet-1.1.0-py3-none-any.whl

Processing /kaggle/input/sequencedata/efficientnet-1.1.0-py3-none-any.whl


In [3]:
IMG_WIDTH=512
IMG_HEIGHT=512
BATCH_SIZE=128
AUTOTUNE = tf.data.experimental.AUTOTUNE 

In [4]:
import efficientnet.tfkeras as efn

In [5]:
enet = efn.EfficientNetB7(
        input_shape=(IMG_WIDTH, IMG_HEIGHT, 3),
        weights=None,
        include_top=False
    )
enet.trainable = True
model = tf.keras.Sequential([
        enet,
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(5, activation='softmax')
    ])
model.compile(
    optimizer='adam',
    loss = 'categorical_crossentropy',
    metrics=['accuracy']
)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
efficientnet-b7 (Model)      (None, 16, 16, 2560)      64097680  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2560)              0         
_________________________________________________________________
dense (Dense)                (None, 5)                 12805     
Total params: 64,110,485
Trainable params: 63,799,765
Non-trainable params: 310,720
_________________________________________________________________


In [6]:
model.load_weights('/kaggle/input/aptos-blindness-detection-augmentation/efficientnetb7epochs36weightswithoversamplingverticalimgaugpart1.hdf5')

In [7]:
path_for_gcs='/kaggle/input/aptos2019-blindness-detection'
print(path_for_gcs)

/kaggle/input/aptos2019-blindness-detection


In [8]:
test_df=pd.read_csv('/kaggle/input/aptos2019-blindness-detection/test.csv')

In [9]:
sample_df=pd.read_csv('/kaggle/input/aptos2019-blindness-detection/sample_submission.csv')
sample_df.head()

,id_code,diagnosis
0,0005cfc8afb6,0
1,003f0afdcd15,0
2,006efc72b638,0
3,00836aaacf06,0
4,009245722fa4,0


In [10]:
test_filenames=list(test_df['id_code'])
test_files_path=[path_for_gcs+'/test_images/' + fname + '.png' for fname in test_filenames ]

In [11]:
def image_aug(img):
    img=tf.image.adjust_gamma(img,gamma=1, gain=1)
    img=tf.image.adjust_contrast(img,1)
    return img

In [12]:
def parse_function(filename):
    image_string=tf.io.read_file(filename)
    image_decoded=tf.image.decode_png(image_string,channels=3)
    #image_decoded=image_aug(image_decoded)
    image_resized=tf.image.resize(image_decoded,[IMG_WIDTH,IMG_HEIGHT])
    image_normalized=image_resized/255.0
    return image_normalized

In [13]:
def create_dataset(filenames):
    dataset=tf.data.Dataset.from_tensor_slices(filenames)
    dataset = dataset.map(parse_function, num_parallel_calls=AUTOTUNE)
    dataset = dataset.prefetch(buffer_size=AUTOTUNE)
    dataset = dataset.batch(BATCH_SIZE)
    return dataset

In [14]:
test_dataset=create_dataset(test_files_path)

In [15]:
probabilities=model.predict(test_dataset)

In [16]:
import numpy as np
predictions=np.argmax(probabilities,axis=-1)
test_df['diagnosis']=predictions

In [17]:
test_df.head()

,id_code,diagnosis
0,0005cfc8afb6,2
1,003f0afdcd15,4
2,006efc72b638,3
3,00836aaacf06,2
4,009245722fa4,2


In [18]:
test_df['diagnosis'].value_counts()

2    1141
0     320
1     203
3     178
4      86
Name: diagnosis, dtype: int64

In [19]:
test_df.to_csv('submission.csv',index=False)